In [ ]:
# 🚀 TELEGRAM API USER ACCOUNT SENDER
# =============================================================================
# CELL 1: Εγκατάσταση
# =============================================================================
!pip install pyrogram tgcrypto pandas openpyxl -q
print("✅ Dependencies εγκαταστάθηκαν!")

# =============================================================================
# CELL 2: User Account Setup
# =============================================================================
from pyrogram import Client
import pandas as pd
import asyncio
import time
from IPython.display import clear_output

API_ID =   # Από my.telegram.org
API_HASH = ""  # Από my.telegram.org
PHONE_NUMBER = ""  # ΤΟ ΔΙΚΟ σας τηλέφωνο

# Έλεγχος credentials
if API_HASH == "your_api_hash_here":
    print("❌ ΒΑΛΤΕ ΤΑ API CREDENTIALS!")
    print("🔗 Από: https://my.telegram.org/")
    print("⚠️ ΠΡΟΣΟΧΗ: User account API, όχι bot token!")
else:
    print("✅ User API credentials έχουν οριστεί!")


# =============================================================================
# CELL 3: Simple User Account Sender
# =============================================================================
class TelegramUserSender:
    def __init__(self, api_id, api_hash, phone_number):
        self.api_id = api_id
        self.api_hash = api_hash
        self.phone_number = phone_number
        # User account client (όχι bot!)
        self.app = Client(
            "user_session",  # Session name
            api_id=api_id,
            api_hash=api_hash,
            phone_number=phone_number
        )

    async def start_session(self):
        """Ξεκινάει user session"""
        print("🔐 Ξεκινάω user session...")
        await self.app.start()

        # User info
        me = await self.app.get_me()
        print(f"✅ Συνδέθηκα ως: {me.first_name} {me.last_name or ''}")
        print(f"📱 Username: @{me.username or 'Χωρίς username'}")
        print(f"📞 Τηλέφωνο: {me.phone_number}")
        print("🎯 User account mode - Full access!")
        return True

    async def send_to_phone(self, phone_number, message):
        """Στέλνει μήνυμα σε τηλέφωνο με contact import fallback"""
        try:
            # Καθαρισμός τηλεφώνου
            if not phone_number.startswith('+'):
                phone_number = f'+{phone_number}'

            # ΣΤΡΑΤΗΓΙΚΗ 1: Άμεση αποστολή
            try:
                await self.app.send_message(phone_number, message)
                return {'success': True, 'error': None}
            except Exception as e1:
                print(f"      ⚠️ Direct send failed: {str(e1)[:50]}")

            # ΣΤΡΑΤΗΓΙΚΗ 2: Προσθήκη στις επαφές πρώτα
            try:
                print("      🔄 Trying contact import...")

                # Import contact using raw API
                from pyrogram.raw.functions.contacts import ImportContacts
                from pyrogram.raw.types import InputPhoneContact

                contact = InputPhoneContact(
                    client_id=0,
                    phone=phone_number.replace('+', ''),
                    first_name="Contact",
                    last_name=""
                )

                result = await self.app.invoke(ImportContacts(contacts=[contact]))

                if result.users:
                    user = result.users[0]
                    await self.app.send_message(user.id, message)
                    return {'success': True, 'error': None}
                else:
                    return {'success': False, 'error': 'Contact import failed - χρήστης δεν βρέθηκε'}

            except Exception as e2:
                print(f"      ⚠️ Contact import failed: {str(e2)[:50]}")

            # ΣΤΡΑΤΗΓΙΚΗ 3: Προσπάθεια με resolve users
            try:
                print("      🔄 Trying user resolution...")
                user = await self.app.get_users(phone_number)
                await self.app.send_message(user.id, message)
                return {'success': True, 'error': None}
            except Exception as e3:
                print(f"      ⚠️ User resolution failed: {str(e3)[:50]}")

            # Αν όλες απέτυχαν
            return {'success': False, 'error': 'Χρήστης δεν μπορεί να λάβει μηνύματα (privacy settings)'}

        except Exception as e:
            error_str = str(e)

            # Ειδικά errors
            if "PEER_ID_INVALID" in error_str:
                return {'success': False, 'error': 'Χρήστης δεν βρέθηκε ή privacy restrictions'}
            elif "USER_IS_BLOCKED" in error_str:
                return {'success': False, 'error': 'Σας έχει μπλοκάρει'}
            elif "CHAT_WRITE_FORBIDDEN" in error_str:
                return {'success': False, 'error': 'Δεν επιτρέπεται μήνυμα (privacy)'}
            elif "FLOOD_WAIT" in error_str:
                import re
                wait_time = re.findall(r'FLOOD_WAIT_(\d+)', error_str)
                wait_seconds = wait_time[0] if wait_time else '60'
                return {'success': False, 'error': f'Rate limit - περιμένετε {wait_seconds}s'}
            else:
                return {'success': False, 'error': error_str}

    async def bulk_send_simple(self, phone_list, message, delay=5):
        """Απλή μαζική αποστολή"""
        results = {
            'total': len(phone_list),
            'sent': 0,
            'failed': 0,
            'blocked': 0,
            'errors': []
        }

        print(f"🚀 Μαζική αποστολή σε {len(phone_list)} τηλέφωνα...")
        print("=" * 50)

        for i, phone in enumerate(phone_list, 1):
            # Progress
            progress = i / len(phone_list) * 100

            clear_output(wait=True)
            print(f"🚀 Αποστολή: {progress:.1f}% ({i}/{len(phone_list)})")
            print(f"📱 Τηλέφωνο: {phone}")
            print(f"✅ Επιτυχή: {results['sent']} | ❌ Αποτυχίες: {results['failed']}")

            # Αποστολή
            result = await self.send_to_phone(phone, message)

            if result['success']:
                results['sent'] += 1
                print("      ✅ Στάλθηκε!")
            else:
                results['failed'] += 1
                error = result['error']
                results['errors'].append(f"{phone}: {error}")
                print(f"      ❌ ΣΦΑΛΜΑ: {error}")

                # Detailed error logging
                if 'PEER_ID_INVALID' in str(error):
                    print("      💡 Χρήστης δεν βρέθηκε - μάλλον δεν έχει Telegram")
                elif 'μπλοκάρει' in error:
                    results['blocked'] += 1
                    print("      🚫 Ο χρήστης σας έχει μπλοκάρει")
                elif 'FLOOD_WAIT' in str(error):
                    print("      ⏳ Rate limit - πρέπει να περιμένετε περισσότερο")

                # Show full error for debugging
                print(f"      🔍 Full error: {error}")

            # Rate limiting (σημαντικό!)
            if i < len(phone_list):
                print(f"      ⏳ Περίμενε {delay} δευτερόλεπτα...")
                await asyncio.sleep(delay)

        # Τελικά αποτελέσματα
        clear_output(wait=True)
        print("🎉 ΟΛΟΚΛΗΡΩΘΗΚΕ!")
        print("=" * 50)
        print(f"📊 ΑΠΟΤΕΛΕΣΜΑΤΑ:")
        print(f"✅ Επιτυχή: {results['sent']}")
        print(f"❌ Αποτυχίες: {results['failed']}")
        print(f"🚫 Blocked by users: {results['blocked']}")
        print(f"📈 Επιτυχία: {results['sent']/results['total']*100:.1f}%")

        if results['errors']:
            print(f"\n❌ DETAILED ERRORS:")
            for error in results['errors'][:5]:
                print(f"   📱 {error}")
            if len(results['errors']) > 5:
                print(f"   ... και {len(results['errors'])-5} ακόμα")

    async def stop_session(self):
        """Τέλος session"""
        await self.app.stop()
        print("🔒 Session τερματίστηκε")

# Δημιουργία user sender
user_sender = TelegramUserSender(API_ID, API_HASH, PHONE_NUMBER)
print("✅ User Account Sender δημιουργήθηκε!")

# =============================================================================
# CELL 4: Test User Connection
# =============================================================================
async def test_user_connection():
    """Δοκιμάζει user account connection"""
    if API_HASH == "your_api_hash_here":
        print("❌ Βάλτε τα credentials!")
        return False

    try:
        await user_sender.start_session()
        print("🎉 User account connection επιτυχής!")
        await user_sender.stop_session()
        return True

    except Exception as e:
        print(f"❌ Connection error: {e}")
        if "phone_number" in str(e).lower():
            print("💡 Βεβαιωθείτε ότι το PHONE_NUMBER είναι σωστό")
        elif "api" in str(e).lower():
            print("💡 Ελέγξτε τα API_ID και API_HASH")
        return False

# Εκτέλεση test
await test_user_connection()

# =============================================================================
# CELL 5: Bulk Send από Excel
# =============================================================================
async def bulk_send_excel():
    """Μαζική αποστολή από Excel"""

    # Upload Excel
    print("📂 Ανεβάστε το Excel:")
    from google.colab import files
    uploaded = files.upload()

    if not uploaded:
        return

    excel_file = list(uploaded.keys())[0]

    try:
        # Read Excel
        df = pd.read_excel(excel_file)
        print(f"📊 Excel columns: {list(df.columns)}")

        # Phone column
        phone_col = input("📱 Phone column name: ").strip()
        if phone_col not in df.columns:
            print(f"❌ Column '{phone_col}' not found!")
            return

        # Clean phones
        phones = df[phone_col].dropna().astype(str).tolist()
        clean_phones = []

        for phone in phones:
            clean = ''.join(filter(str.isdigit, phone))
            if len(clean) >= 10:
                if clean.startswith('69') and len(clean) == 10:
                    clean_phones.append(f'+30{clean}')
                elif clean.startswith('3069'):
                    clean_phones.append(f'+{clean}')
                else:
                    clean_phones.append(f'+{clean}')

        print(f"📱 Valid phones: {len(clean_phones)}")

        # Message
        MESSAGE = input("💬 Your message: ").strip() or """🎉 Special Offer!

Hello! We have a great offer for you:
✨ 50% discount on all products
⏰ Limited time only!

Visit: https://www.yourstore.com

Thank you! 🙏"""

        print(f"\n📋 SUMMARY:")
        print(f"   📱 Phones: {len(clean_phones)}")
        print(f"   💬 Message: {MESSAGE[:50]}...")

        # Confirmation
        confirm = input(f"\n❓ Send to {len(clean_phones)} phones? (yes/no): ")

        if confirm.lower() in ['yes', 'y']:
            print("🚀 Starting bulk send...")

            # Start session
            await user_sender.start_session()

            # Send
            results = await user_sender.bulk_send_simple(clean_phones, MESSAGE)

            # Stop session
            await user_sender.stop_session()

            print(f"\n🎊 DONE! Sent {results['sent']} messages successfully!")

        else:
            print("❌ Cancelled")

    except Exception as e:
        print(f"❌ Error: {e}")

# Run bulk send
await bulk_send_excel()

# =============================================================================
# CELL 6: Single Test Message
# =============================================================================
async def single_test():
    """Test με ένα τηλέφωνο"""

    test_phone = input("📱 Test phone (+306912345678): ").strip()
    test_msg = "🧪 Test message από User Account API!"

    if test_phone:
        print(f"📱 Sending to: {test_phone}")

        await user_sender.start_session()
        result = await user_sender.send_to_phone(test_phone, test_msg)
        await user_sender.stop_session()

        if result['success']:
            print("✅ Test successful!")
        else:
            print(f"❌ Test failed: {result['error']}")

🎉 ΟΛΟΚΛΗΡΩΘΗΚΕ!
📊 ΑΠΟΤΕΛΕΣΜΑΤΑ:
✅ Επιτυχή: 3
❌ Αποτυχίες: 0
🚫 Blocked by users: 0
📈 Επιτυχία: 100.0%
🔒 Session τερματίστηκε
❌ Error: 'NoneType' object is not subscriptable


# Telegram User Account Sender Documentation

This project provides a Python script to send messages from a Telegram user account to phone numbers, supporting both single message sending and bulk sending from an Excel file.

## Features

-   **User Account Integration**: Connects to Telegram using a user's API ID, API Hash, and phone number.
-   **Single Message Send**: Send a message to a specific phone number.
-   **Smart Contact Handling**: Attempts to send messages directly, and if that fails, tries to import the contact and then send, or resolve the user.
-   **Bulk Sending from Excel**: Reads phone numbers from an Excel file and sends a predefined message to all valid numbers with configurable delays.
-   **Rate Limit Handling**: Includes basic rate-limiting awareness with configurable delays between messages to avoid flood waits.

## Setup

### 1. Install Dependencies

Run the first cell in the notebook to install all necessary libraries:

```python
!pip install pyrogram tgcrypto pandas openpyxl -q
```

### 2. Configure API Credentials

Update `CELL 2` with your Telegram API credentials. You can obtain these from [my.telegram.org](https://my.telegram.org/). Make sure to use your **user account API** credentials, not a bot token.

-   `API_ID`: Your API ID.
-   `API_HASH`: Your API Hash.
-   `PHONE_NUMBER`: Your Telegram phone number (e.g., `+3069XXXXXXXX`).

```python
API_ID =  # Replace with your API ID
API_HASH = "" # Replace with your API Hash
PHONE_NUMBER = "" # Replace with your phone number
```

**Important**: If `API_HASH` is still `"your_api_hash_here"`, the script will prompt you to update it.

## Usage

### 1. Test User Account Connection

Run `CELL 4` (`test_user_connection()`) to verify that your API credentials are correct and that the script can successfully connect to your Telegram account.

```python
await test_user_connection()
```

### 2. Send a Single Test Message

To send a single message to a specific phone number, run `CELL 6` (`single_test()`). You will be prompted to enter a test phone number.

```python
await single_test()
```

### 3. Bulk Send Messages from Excel

To send messages to multiple phone numbers from an Excel file, run `CELL 5` (`bulk_send_excel()`).

**Expected Excel Format**:

The Excel file should contain a column with phone numbers. The column name will be specified during execution.

**Steps:**

1.  Run the `bulk_send_excel()` function.
2.  You will be prompted to upload your Excel file.
3.  After uploading, the script will display the columns found in your Excel file. You will then be asked to enter the exact name of the column containing the phone numbers.
4.  You will be prompted to enter the message you wish to send. A default message is provided if you leave it blank.
5.  A summary of the operation will be displayed, and you will be asked for confirmation before sending the messages.
6.  The script will then proceed with sending messages, showing progress and detailed results.

```python
await bulk_send_excel()
```

## Error Handling & Notes

-   **Rate Limits (`FLOOD_WAIT`)**: Telegram imposes rate limits. The `bulk_send_simple` function includes a delay between messages to mitigate this. If you encounter `FLOOD_WAIT` errors, consider increasing the `delay` parameter in the `bulk_send_simple` method.
-   **Privacy Settings**: Some users may have privacy settings that prevent direct messages from non-contacts. The script attempts workarounds (contact import), but some messages may still fail due to user privacy settings.
-   **Invalid Phone Numbers**: The script attempts to clean phone numbers (e.g., adding `+30` for Greek numbers), but ensures your Excel data is as clean as possible.
-   **Session Management**: The script starts and stops the Pyrogram session for each operation (`start_session()` and `stop_session()`). This ensures resources are managed correctly.